# Tutorial 10: A/B Testing and principled peeking

## Learning Objectives

After completing this week's worksheet and tutorial work, you will be able to:

1. Explain sequential testing and principled peeking and how it can be used for early stopping of an experiment (e.g., A/B testing).
2. Write a computer script to perform A/B testing optimization with and without using principled peeking.
3. Discuss the tradeoff between stopping earlier and certainty of significance, and their real-world implications.
4. List methods used to correct for inflated error rates.

In [ ]:
# Run this cell before continuing.
library(tidyverse)
library(gsDesign)

source("tests_tutorial_10.R")

## 1. Warm up questions

**Question 1.0**
<br>{points: 1}

Sequential A/B testing is used to analyze only continuous variables. **True or False?**

*Assign your answer to an object called answer1.0. Your answer should be either "true" or "false", surrounded by quotes.*

In [ ]:
#answer1.0 <- 

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_1.0()

**Question 1.1**
<br>{points: 1}

When performing sequential A/B testing, a power analysis is not required since the analyst will check the data before collecting all data. **True or False?**

*Assign your answer to an object called answer1.1. Your answer should be either "true" or "false", surrounded by quotes.*

In [ ]:
#answer1.1 <- 

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_1.1()

**Question 1.2**
<br>{points: 3}

In **full sequential designs**, the analyst performs an analysis after every new observation, sequentially. Would you recommend using a Bonferroni correction for **full sequential designs** of large experiments so that it can be stopped as soon as possible?

*Briefly, justify your recommendation. Think of pros and cons of the correction for the experiment described.*

DOUBLE CLICK TO EDIT **THIS CELL** AND REPLACE THIS TEXT WITH YOUR ANSWER.

## 2. Early peeking in A/B testing

In worksheet_10, we studied by means of an A/A testing (i.e., the case where we know that there's no difference in the distributions from the groups) how peeking can inflate the probability of Type I Error. 

In addition, we used different methods to implement principled peeking and early stopping rules in sequential A/B testing.

- **Bonferroni's method** provides an adjustment to classical $p$-values (or equivalently the significance level or critical values) to control the type I error rate. 

- **Pocock's method**, available in `gsDesign` offers a less conservative way of controlling the type I error rate in sequential testing with early stops.

- These 2 methods used uniform boundaries (common critical values) for all interim tests.

In this section, you will examine how the number of interim tests may affect the results and the boundary used. 

**Question 2.0**
<br>{points: 1}

Use the package `gsDesign` to get critical values for the **Pocock's method** for the following experimental design:

- one-sided test to compare two population means (recall that the package will implement $z$-tests which are similar to $t$-tests when sample size is large)

- 20 sequential (interim) tests

- A/A testing design, i.e., effect size = 0 

- a significance level of $5\%$ 

- a power of $80\%$

*Assign your final answer to an object called `crit_pocock_20`. Your solution should be a vector with 20 equal values.*

In [ ]:
# design_pocock <- gsDesign(k = ..., #number of interim analyses planned
#                          test.type = 1, # for one-sided tests
#                          delta = ... , # default effect size
#                          alpha = ..., #type I error rate
#                          beta = ..., # type II error rate
#                          sfu = 'Pocock')

# crit_pocock_20 <- design_pocock$upper$bound

# your code here
fail() # No Answer - remove if you provide an answer

crit_pocock_20

In [ ]:
test_2.0()

**Question 2.1**
<br>{points: 1}

Repeat **Question 2.0** but this time get Pocock's critical values for an experimental design that will peek at the data 10 times (i.e., 10 sequential (interim) tests).

*Write the appropriate code and assign your answer to an object called `crit_pocock_10`. Your solution should be a vector with 10 equal values.*

In [ ]:
# Your code goes here. No skeleton code is provided; see the previous question.

# your code here
fail() # No Answer - remove if you provide an answer

crit_pocock_10

In [ ]:
test_2.1()

**Question 2.2**
<br>{points: 3}

The Pocock's critical values obtained in **Question 2.0** and **Question 2.1** show that as the number of peeks (interim tests) increases, the critical values also increase. 

Briefly, explain why.

DOUBLE CLICK TO EDIT **THIS CELL** AND REPLACE THIS TEXT WITH YOUR ANSWER.

**Question 2.3**
<br>{points: 1}

Suppose that another company decides to use the same test, the same significance level and also to peek 10 times. If the new company wants to increase the probability of finding a significance result when $H_0$ is false (i.e., have more power) and still control the type I error rate, which of the following strategies would you recommend:

**A** Use the same Pocock's critical value as in **Question 2.1** to control the type I error and plan for a larger experiment (i.e., larger sample size)

**B** Don't use the Pocock's method since it's too conservative. Just use raw $p$-values from the CLT sampling distribution.

**C** Use a Bonferroni's correction

**D** There is no way to increase the power of the test

*Hint*: you can use code to design this new experiment

*Assign your answer to an object called answer2.3. Your answer should be one of `"A"`, `"B"`, `"C"`, or `"D"` surrounded by quotes.*

In [ ]:
# answer2.3 <- 

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_2.3()

## 3. Principled peeking: O’Brien-Fleming method

In this section, we will implement and examine another method, also available in `gsDesign`: the **O’Brien-Fleming method**

Unlike previous methods covered in worksheet_10, the O’Brien-Fleming method uses *non-uniform* boundaries, which has conservative critical values for earlier interim analyses and less conservative values (closer to the unadjusted critical values) as more data are collected. 

As in **Question 3.2.1 of worksheet_10** we will use the function `incremental_t_test` to generate and sequentially analyze data.

In the next few exercises, we will plot the statistics of the sequential tests and add 4 type of boundaries:

- the unadjusted critical values (black line)

- the Bonferroni's adjusted critical values (blue line)

- the Pocock critical values (red line)

- the O'Brien-Fleming critical values (green line)

*Run the following cell to get the function to simulate and analyze data of sequential A/B testing*

In [ ]:
# Two-sample t-test with tracking sequential statistic and p-values by incremental sample sizes until getting to n.

# @param n (numeric): Initially planned sample size for each group (for simplicity, n needs to be a multiple of sample_increase_step).
# @param d_0 (numeric): effect size.
# @param mean_current (numeric): Population mean for control variation.
# @param sd_current (numeric): Population standard deviation for current variation.
# @param sd_new (numeric): Population standard deviation for new variation.
# @param sample_increase_step (numeric): Sample size increment.

# @return p.value.df: A tibble that has 3 columns:
# inc_sample_size, statistic, and p_value 

incremental_t_test <- function(n, d_0, mean_current, sd_current, sd_new, sample_increase_step) {
  sample_current <- rnorm(n, mean = mean_current, sd = sd_current)
  sample_new <- rnorm(n, mean = mean_current + d_0, sd = sd_new)

  p.value.df <- tibble(
    inc_sample_size = rep(0, n / sample_increase_step),
    statistic = rep(0, n / sample_increase_step),
    p_value = rep(0, n / sample_increase_step)
  )

  current_sample_size <- sample_increase_step
  
  for (i in 1:nrow(p.value.df))
  {
    t_test_results <- t.test(sample_new[1:current_sample_size], sample_current[1:current_sample_size],
      var.equal = TRUE,
      alternative = "greater"                      
    )
    p.value.df[i, "statistic"] <- as_tibble(t_test_results$statistic)
    p.value.df[i, "p_value"] <- as_tibble(t_test_results$p.value)
    p.value.df[i, "inc_sample_size"] <- current_sample_size
    current_sample_size <- current_sample_size + sample_increase_step
  }

  return(p.value.df)
}

**Question 3.0**
{points: 1}

**A/A testing**: as in worksheet_10, let's simulate data that reflects no difference in the population means (i.e., $H_0$ is true).

In this question, analyze the data in batches of 100 experimental units per group until a total of $n = 1000$ per group is collected (i.e., plan for 10 sequential tests).
  
We will assume again that data of both groups is generated from a Normal distribution with a mean equal to \\$200 and a standard deviation equal to \\$50.

Use the `incremental_t_test` function to conduct this experiment.

_Save the result in an object called answer3.0. Your answer should be a tibble with three columns: `inc_sample_size`, `statistic`, and `p_value`._

In [ ]:
set.seed(25) # do not change this.

# answer3.0 <- 
#    incremental_t_test(n = ..., d_0 = ..., sample_increase_step = ..., mean_current = ..., sd_current = ..., sd_new = ...)

# your code here
fail() # No Answer - remove if you provide an answer

answer3.0

In [ ]:
test_3.0()

**Question 3.1**
<br>{points: 3}

Repeat **Question 2.0** but this time get O'Brien-Fleming's critical values for an experimental design that will peek at the data 10 times (i.e., 10 sequential (interim) tests).

*Write the appropriate code and assign your answer to an object called `crit_of_10`. Your solution should be a vector with 10 numeric values.*

In [ ]:
# Get critical values of the O'Brien-Fleming design!

# Your code goes here. No skeleton code is provided; see question 2.0.

# your code here
fail() # No Answer - remove if you provide an answer

crit_of_10

In [ ]:
# Here we check to see if you have given your answer the correct object name
# and if your answer is plausible. However, all other tests have been hidden
# so you can practice deciding when you have the correct answer.

test_that('Did not assign answer to an object called "crit_of_10"', {
    expect_true(exists("crit_of_10"))
})

test_that("Solution should be a vector of length 10", {
    expect_equal(digest(length(crit_of_10)), "71db8a6cad03244e6e50f0ad8bc95a65")
})


**Question 3.2**
<br>{points: 1}

Using the data stored in `answer3.0`, plot the sequence of observed statistics for each interim analysis as a **line** with the incremental sample size on the $x$-axis and the value of the observed statistic on the $y$-axis. 

Add 4 dashed lines that indicate the following 4 boundaries (critical values): 

- a green line for the OF's critical values

- a red line for the Pocock's critical values

- a blue line for the Bonferroni's critical values

- a black line for the unadjusted critical values

The `ggplot()` object's name will be `sequential_stat`.

*Fill out those parts indicated with `...`, uncomment the corresponding code in the cell below, and run it.*

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 9) # Adjust these numbers so the plot looks good in your desktop.

#crit_unadj <- qt(1 - ..., ...)
#crit_bonferroni <- ...(1 - ..., ...)

#sequential_stat <- 
#  answer3.0 %>%
#  ggplot() +
#  geom_line(aes(x = inc_sample_size, y = statistic)) +
#  geom_point(aes(x = ..., y = ...)) +
#  geom_line(aes(x = inc_sample_size, y = crit_of_10),colour = 3, linetype = "twodash")+
#  geom_point(aes(x = inc_sample_size, y = ...), colour = 3) +
#  geom_text(x=150, y=crit_of_10[1] + 0.15, size=6, label="O'Brien-Fleming",colour = 3) +
#  geom_hline(yintercept = ..., colour = "red", linetype = "twodash") +
#  geom_point(aes(x = inc_sample_size, y = ...), colour = "red") +
#  geom_text(x=150, y=crit_pocock_10 + 0.15, size=6, label="Pocock",colour = "red") +
#  geom_hline(yintercept = ..., colour = "blue", linetype = "twodash") +
#  geom_point(aes(x = inc_sample_size, y = rep(crit_Bonferroni, ...)), colour = "blue") +
#  geom_text(x=150, y=crit_bonferroni + 0.15, size=6, label="Bonferroni",colour = "blue") +
#  geom_hline(yintercept = ..., linetype = "twodash") +
#  geom_point(aes(x = inc_sample_size, y = rep(..., ...))) +
#  geom_text(x=150, y=crit_unadj + 0.15, size=6, label="Unadjusted") +
#  theme(
#    text = element_text(size = 18),
#    plot.title = element_text(face = "bold"),
#    axis.title = element_text(face = "bold")
#  ) +
#  ggtitle("Critical values in Sequential Designs") +
#  ylab("Statistic") +
#  xlab("Sample Size") +
#  coord_cartesian(ylim = c(-1, 6)) +
#  scale_y_continuous(breaks = seq(-1, 6, by = 0.5))

# your code here
fail() # No Answer - remove if you provide an answer

sequential_stat

In [ ]:
test_3.2()

**Question 3.3**
<br>{points: 3}

Suppose that the generated data correspond to the political campaign experiment and that the organizers have decided to monitor the data every 100 visitors per website and stop the experiment earlier if there's evidence of a difference between the group means. 

According to the data plotted in **Question 3.2**, is the following statement **True or False?**

> The campaign organizers will not erroneously stop the experiment if they compare the observed statistics with any of the boundaries that control the type I error rate

*Assign your answer to an object called `answer3.3`. Your answer should be either `"true"` or `"false"`, surrounded by quotes.*

In [ ]:
# answer3.3 <- 

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
# Here we check to see if you have given your answer the correct object name
# and if your answer is plausible. However, all other tests have been hidden
# so you can practice deciding when you have the correct answer.

test_that('Did not assign answer to an object called "answer3.3"', {
    expect_true(exists("answer3.3"))
})

test_that('Solution should be "true" or "false"', {
    expect_match(answer3.3, "true|false", ignore.case = TRUE)
})
  


**Question 3.4**
<br>{points: 1}

To examine how the O'Brien-Fleming (OF) controls the type I error rate, the campaign organizers decided to: 

- perform the **A/A testing** experiment 100 times 

- count how many times they would wrongly reject $H_0$ with their strategy, and

- compare it with the expected number of rejections given the significance level $\alpha = 0.05$

Use the code below to run 100 experiments and then estimate the type I error rate for the OF method.

Your answer will be a tibble with three columns: `n_rejections_OF`, `n_rejections_unadj`, and `expected_n_rejections`.

These columns should contain: the number of wrong rejections among the 100 experiments for the OF and the classical methods (unadjusted), respectively, compared to the expected number of wrong rejections given the design.

*Fill out those parts indicated with `...`, uncomment the corresponding code in the cell below, and run it.*

In [ ]:
set.seed(120)

### Run this before continuing
multiple_times_sequential_tests <- 
    tibble(experiment = 1:100) %>% 
    mutate(seq_test = map(.x = experiment, 
                          .f = function(x) incremental_t_test(n = 1000, d_0 = 0, sample_increase_step = 100, 
                              mean_current = 200, sd_current = 50, sd_new = 50)))

In [ ]:
#answer3.4 <- multiple_times_sequential_tests %>% 
#    mutate(reject_of = map_dbl(.x = seq_test, .f = function(x) sum(... ... ...) > 0),
#           reject_unadj = map_dbl(.x = seq_test, .f = function(x) sum(... ... ...) >0)) %>%  
#    summarise(n_rejections_OF = ...(reject_of),
#              n_rejections_unadj = ...(reject_unadj),
#              expected_n_rejections = ...)

# your code here
fail() # No Answer - remove if you provide an answer
                            
answer3.4

In [ ]:
test_3.4()

**Question 3.5**
<br>{points: 3}

Explain briefly the results obtained in **Question 3.4**.


DOUBLE CLICK TO EDIT **THIS CELL** AND REPLACE THIS TEXT WITH YOUR ANSWER.